In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
import json
import os
import re
import time
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
cookies = {
    'cookiesession1': '678B28F695C90B262BFC00B79BBD6CC1',
    'lhc_per': 'vid|a3d85efe963899fee2b5',
    '_gid': 'GA1.3.289911229.1685154797',
    '_ga': 'GA1.1.2075372866.1684772919',
    '_ga_3C534XGNC2': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_Q33BVTTDL1': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
    '_ga_QHEB12PC6D': 'GS1.1.1685154797.4.1.1685154882.0.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'cookiesession1=678B28F695C90B262BFC00B79BBD6CC1; lhc_per=vid|a3d85efe963899fee2b5; _gid=GA1.3.289911229.1685154797; _ga=GA1.1.2075372866.1684772919; _ga_3C534XGNC2=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_Q33BVTTDL1=GS1.1.1685154797.4.1.1685154882.0.0.0; _ga_QHEB12PC6D=GS1.1.1685154797.4.1.1685154882.0.0.0',
    'Pragma': 'no-cache',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

In [5]:
def download(url, file_name, retry = 3):
    if 'um.edu.my' in url:
        return
    if os.path.exists(file_name) and os.path.getsize(file_name) > 50000:
        return
    with open(file_name, "wb") as file:
        for k in range(retry):
            try:
                response = requests.get(url, verify = False, timeout = 360, cookies=cookies, headers=headers)
                file.write(response.content)
                return 
            except Exception as e:
                print(k, e)
                time.sleep(0.5)

In [6]:
files = glob('pages-uia/*.json')
len(files)

5923

In [7]:
pdfs = []
for f in files:
    with open(f) as fopen:
        pdfs.extend(json.load(fopen))
        
pdfs = list(set(pdfs))

In [8]:
filtered_pdfs = []
for p in tqdm(pdfs):
#     if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
#         continue
#     if not p.endswith('.pdf'):
#         continue
#     if 'ade.sagepub.com' in p:
#         continue
    
#     if p.startswith('/jspui'):
#         p = 'https://oarep.usim.edu.my' + p
    
    if len(re.findall( r'[0-9]+(?:\.[0-9]+){3}', p)):
        continue
    if not p.endswith('.pdf'):
        continue
        
    if not p.startswith('http'):
        continue
        
    filtered_pdfs.append(p)
    
filtered_pdfs = sorted(list(set(filtered_pdfs)))
len(filtered_pdfs)

100%|█████████████████████████████████| 87906/87906 [00:00<00:00, 348617.50it/s]


87736

In [9]:
# filtered_pdfs

In [10]:
uia = filtered_pdfs

In [11]:
# !rm -rf usim
!mkdir uia

mkdir: cannot create directory ‘uia’: File exists


In [14]:
max_worker = 1

uia = sorted(list(uia))
for i in tqdm(range(0, len(uia), max_worker)):
    urls = [(url, os.path.join('uia', f'{i}-{no}.pdf')) for no, url in enumerate(uia[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            future.result()

  0%|                                                 | 0/87736 [00:00<?, ?it/s]

0 HTTPConnectionPool(host='abqarijournal.usim.edu.my', port=80): Max retries exceeded with url: /images/Volume/Volume_13/5.%2049-58.%20ZAKAT%20PENDAPATAN%20ALTERNATIF%20CUKAI%20PENDAPATAN.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f47aa1a1340>: Failed to establish a new connection: [Errno 110] Connection timed out'))
0 HTTPConnectionPool(host='abqarijournal.usim.edu.my', port=80): Max retries exceeded with url: /images/Volume/Volume_13/5.%2049-58.%20ZAKAT%20PENDAPATAN%20ALTERNATIF%20CUKAI%20PENDAPATAN.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f47aa146790>: Failed to establish a new connection: [Errno 110] Connection timed out'))
1 HTTPConnectionPool(host='abqarijournal.usim.edu.my', port=80): Max retries exceeded with url: /images/Volume/Volume_13/5.%2049-58.%20ZAKAT%20PENDAPATAN%20ALTERNATIF%20CUKAI%20PENDAPATAN.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f47aa155130

  0%|                                   | 2/87736 [06:32<4788:31:08, 196.49s/it]

0 HTTPConnectionPool(host='abqarijournal.usim.edu.my', port=80): Max retries exceeded with url: /images/Volume/Volume_8/Usage%20and%20Satisfaction.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f47aa146b80>: Failed to establish a new connection: [Errno 110] Connection timed out'))
2 HTTPConnectionPool(host='abqarijournal.usim.edu.my', port=80): Max retries exceeded with url: /images/Volume/Volume_8/Usage%20and%20Satisfaction.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f47aa20bc10>: Failed to establish a new connection: [Errno 110] Connection timed out'))


  0%|                                   | 3/87736 [13:06<6786:07:52, 278.46s/it]

...

100%|█████████████████████████████████▉| 87467/87736 [11:14:31<19:07,  4.26s/it]

0 HTTPSConnectionPool(host='www.journal.atmph-specialissues.org', port=443): Max retries exceeded with url: /uploads/179/7615_pdf.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f47a86bed90>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPSConnectionPool(host='www.journal.atmph-specialissues.org', port=443): Max retries exceeded with url: /uploads/179/7615_pdf.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f47a86beb50>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPSConnectionPool(host='www.journal.atmph-specialissues.org', port=443): Max retries exceeded with url: /uploads/179/7615_pdf.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f47a86bea90>: Failed to establish a new connection: [Errno -2] Name or service not known'))


100%|█████████████████████████████████▉| 87484/87736 [11:15:36<21:06,  5.03s/it]

0 HTTPSConnectionPool(host='www.ki2m.my', port=443): Max retries exceeded with url: /wp-content/uploads/2022/07/e-ISBN-E-PROCEEDING-KI2M2022.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f47a86bea90>: Failed to establish a new connection: [Errno -2] Name or service not known'))
1 HTTPSConnectionPool(host='www.ki2m.my', port=443): Max retries exceeded with url: /wp-content/uploads/2022/07/e-ISBN-E-PROCEEDING-KI2M2022.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f47a86bea90>: Failed to establish a new connection: [Errno -2] Name or service not known'))
2 HTTPSConnectionPool(host='www.ki2m.my', port=443): Max retries exceeded with url: /wp-content/uploads/2022/07/e-ISBN-E-PROCEEDING-KI2M2022.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f47a86be100>: Failed to establish a new connection: [Errno -2] Name or service not known'))


100%|██████████████████████████████████| 87736/87736 [11:36:49<00:00,  2.10it/s]
